In [1]:
import pandas as pd
import numpy as np
path = ''
import gc
from sklearn.ensemble import ExtraTreesRegressor 
#from sklearn.cross_validation import train_test_split 
from sklearn.metrics import mean_squared_error as rmse
from sklearn.cross_validation import KFold
import sys
# sys.path.append(path+'/src/python/')
from data_load import loadHDF_train
from data_load import loadHDF_test
from time import time
from sys import stdout
from copy import deepcopy

seed = 3555
np.random.seed(seed)

kfold=5
params = {"criterion": "mse", 
          "max_features": 0.7,
          "max_depth": 6,
          "min_samples_split": 100,  
          "min_weight_fraction_leaf": 0}
num_round = 20
clip = [1.,50.]
unit = 3600.*24.

outputfilehead = 'ext_dM1-1hrAgg.v3.CV5.md6'
finalsubmitfile = '0916_'+outputfilehead+'.havg.csv'
datafilehead = path+'feat_1hr.v3.'

print(num_round)
print(params)

colnames = [u'meanInv.custid', u'q25Inv.custid',
       u'q50Inv.custid', u'q75Inv.custid', u'q95Inv.custid', u'q999Inv.custid',
       u'rInvGT1.custid', u'rInvGT10.custid', u'rInvGT2.custid',
       u'cnt.eventid', u'q25Inv.eventid', u'q50Inv.eventid', u'q75Inv.eventid',
       u'q95Inv.eventid', u'rInvGT1.eventid', u'rInvGT10.eventid',
       u'rInvGT2.eventid', u'meanInv.curHr', u'q75Inv.curHr', u'q95Inv.curHr',
       u'q999Inv.curHr', u'rInvGT10.curHr', u'rInvGT2.curHr',
       u'meanInv.pltfHr', u'q75Inv.pltfHr', u'q95Inv.pltfHr',
       u'q999Inv.pltfHr', u'rInvGT10.pltfHr', u'rInvGT2.pltfHr',
       u'meanInv.grpidHr', u'q75Inv.grpidHr', u'q95Inv.grpidHr',
       u'q999Inv.grpidHr', u'rInvGT10.grpidHr', u'rInvGT2.grpidHr']
#didn't use
month=6
timelimit = 3600.*24* np.array([(month-1),month])*30 
np.random.seed(seed)
np.random.randint(0,1000)

20
{'max_features': 0.7, 'min_samples_split': 100, 'min_weight_fraction_leaf': 0, 'criterion': 'mse', 'max_depth': 6}


713

In [2]:
def preprocess(d, timelimit=[0,999999999], missing=0, index = []):
#    if(len(index) > 0):
#        index = index[(d['timestamp'] >= timelimit[0]) & (d['timestamp'] < timelimit[1])]

#    dcp = deepcopy(d[(d['timestamp'] >= timelimit[0]) & (d['timestamp'] < timelimit[1])])
#    del d
#    gc.collect()
#    d.drop('timestamp', inplace=True, axis=1)
    d.fillna(missing, inplace=True)
    if(len(index) == 0):
        return d
    else:
        return d, index

In [3]:
## redo the training for train2, test2
dtr2, pred2, dtr2_index = loadHDF_train(datafilehead+'train.h5', key='train', 
                                     columns = colnames)

In [4]:
## preprocess
dtr2, dtr2_index = preprocess(dtr2, missing=0, index=dtr2_index)
pred2 = preprocess(pred2, missing=0)

In [5]:
dte2, dte2_index = loadHDF_test(datafilehead+'test.h5', key='test', columns = colnames)
dte2 = preprocess(dte2, missing=0)
num_te2 = dte2.shape[0]
fmap = dte2.columns

In [6]:
kf = KFold(pred2.shape[0], n_folds=kfold, random_state=np.random.randint(0,1000))
ptest = pd.DataFrame({'invest': np.zeros(num_te2)},index=dte2_index)
score_list = []
sc_val = []
val_result = pd.DataFrame()
featImp = pd.DataFrame({"gain": np.zeros(len(fmap))}, index = fmap)
featImp.sort_index(inplace=True)
for k, (tr_index, v_index) in enumerate(kf):
    train_index = dtr2_index[tr_index]
    val_index = dtr2_index[v_index]

    m_start = time()
    X_train, X_val = dtr2.loc[train_index], dtr2.loc[val_index]
    y_train, y_val = pred2.loc[train_index].values[:,0], pred2.loc[val_index].values[:,0]
    X_valIndex = X_val.index

    model = ExtraTreesRegressor(n_estimators = num_round, n_jobs=-1,\
                                random_state = np.random.randint(0,1000))
    model.set_params(**params)
    model.fit(X_train, y_train)
    
    pred_train = model.predict(X_train)
    pred_val = model.predict(X_val)
    pred_test = model.predict(dte2)

    pred_train = np.clip(pred_train, clip[0], clip[1])
    pred_val = np.clip(pred_val, clip[0], clip[1])
    pred_test = np.clip(pred_test, clip[0], clip[1])
    
    m_end = time()

    featImp0 = pd.DataFrame({"gain": model.feature_importances_},index=fmap)
    featImp['gain'] = featImp0['gain']/kfold + featImp['gain']
                            
    score_train = rmse(y_train, pred_train)
    score_val = rmse(y_val, pred_val)
    dd = pd.DataFrame({'pred':pred_val,'invest':y_val, 
                       'fold':(np.zeros(y_val.shape[0])+k)}, 
                      index=X_valIndex)

    ptest = pd.concat([ptest, pd.DataFrame({k:1./pred_test},index=dte2_index)],axis=1) 
    
    val_result = pd.concat([val_result, dd])
    score_list.append({'fold':k,'score_valid': score_val, 
                       'score_train': score_train, 'time':m_end-m_start})
    sc_val.append(score_val)

    print "fold:{0}, train={1:0.3f}, val={2:0.3f}, val1={3:0.3f}, val2={4:0.3f},\
    proc_time={5:5.0f}".format(k,score_train, score_val,0, 0, m_end-m_start)
    stdout.flush()
    del y_train, y_val, X_train, X_val, pred_train, pred_val, pred_test, model
    gc.collect()

fold:0, train=0.847, val=1.050, val1=0.000, val2=0.000,    proc_time=   52
fold:1, train=0.875, val=0.996, val1=0.000, val2=0.000,    proc_time=   51
fold:2, train=0.942, val=0.714, val1=0.000, val2=0.000,    proc_time=   54
fold:3, train=0.901, val=0.973, val1=0.000, val2=0.000,    proc_time=   57
fold:4, train=0.820, val=1.474, val1=0.000, val2=0.000,    proc_time=   51


In [7]:
columns = ptest.columns[1:]
ptest['invest'] = kfold/(ptest[columns].sum(axis=1))    
ptest = ptest['invest']

## save validate to file
ptest.to_hdf(path+outputfilehead+'.te.h5','test', format='table')
val_result.to_hdf(path+outputfilehead+'.tr.h5','model',format='table')

score_list = pd.DataFrame(score_list)
    
totalTime = score_list['time'].sum() / 60.
meanLCV = score_list['score_valid'].mean()
stdLCV = score_list['score_valid'].std()

print('num_round: {0}'.format(num_round))
print(params)
print('## Summary: score_valid mean = {0:0.5f} pm {1:0.5f} with total process time: {2:5.0f}'.\
      format(meanLCV, stdLCV, totalTime))
stdout.flush()

## write out submission file
ptest.to_csv(path+finalsubmitfile,header=False,index=False)

print('## feature importance')
featImp.sort_values(by=['gain'],axis=0, ascending=False, inplace=True)
print(featImp.head(10))
featImp.to_csv(path+outputfilehead+'.extfeatImp.3.csv')

del dtr2, dtr2_index, pred2, dte2_index
gc.collect()

num_round: 20
{'max_features': 0.7, 'min_samples_split': 100, 'min_weight_fraction_leaf': 0, 'criterion': 'mse', 'max_depth': 6}
## Summary: score_valid mean = 1.04133 pm 0.27470 with total process time:     4
## feature importance
                     gain
q95Inv.custid    0.184563
meanInv.custid   0.152117
rInvGT10.custid  0.129977
q75Inv.custid    0.111409
q50Inv.custid    0.098107
rInvGT2.custid   0.067642
q25Inv.custid    0.051358
q75Inv.grpidHr   0.040239
q999Inv.custid   0.039388
rInvGT1.custid   0.029353


140